In [1]:
import numpy as np
import pandas as pd
from scipy import ndimage
import matplotlib.pyplot as plt

def update_path(source_path, new_path_prefix):
    filename = source_path.split('/')[-1]
    return new_path_prefix + filename

driving_log = pd.read_csv('data/driving_log.csv')

In [2]:
images = []
steering_angles = []

high_steering = (driving_log.steering < -0.25) | (driving_log.steering > 0.25)

for img_path_orig, steering_angle in zip(driving_log['center'], driving_log['steering']):
    img_path = update_path(img_path_orig, 'data/IMG/')
    image = plt.imread(img_path)
    images.append(image)
    steering_angles.append(steering_angle)
    images.append(np.fliplr(image))
    steering_angles.append(-steering_angle)
    
for img_path_orig, steering_angle in zip(driving_log['left'], driving_log['steering']):
    img_path = update_path(img_path_orig, 'data/IMG/')
    image = plt.imread(img_path)
    images.append(image)
    steering_angle += 0.25
    steering_angles.append(steering_angle)
    images.append(np.fliplr(image))
    steering_angles.append(-steering_angle)
    
for img_path_orig, steering_angle in zip(driving_log['right'], driving_log['steering']):
    img_path = update_path(img_path_orig, 'data/IMG/')
    image = plt.imread(img_path)
    images.append(image)
    steering_angle -= 0.25
    steering_angles.append(steering_angle)
    images.append(np.fliplr(image))
    steering_angles.append(-steering_angle)
    
# for img_path_orig, steering_angle in zip(driving_log['center'][high_steering], driving_log['steering'][high_steering]):
#     img_path = update_path(img_path_orig, 'data/IMG/')
#     image = plt.imread(img_path)
#     images.append(image)
#     steering_angles.append(steering_angle)
#     images.append(np.fliplr(image))
#     steering_angles.append(-steering_angle)
#     images.append(image)
#     steering_angles.append(steering_angle)
#     images.append(np.fliplr(image))
#     steering_angles.append(-steering_angle)
    
# for img_path_orig, steering_angle in zip(driving_log['left'][high_steering], driving_log['steering'][high_steering]):
#     img_path = update_path(img_path_orig, 'data/IMG/')
#     image = plt.imread(img_path)
#     steering_angle += 0.25
#     images.append(image)
#     steering_angles.append(steering_angle)
#     images.append(np.fliplr(image))
#     steering_angles.append(-steering_angle)
#     images.append(image)
#     steering_angles.append(steering_angle)
#     images.append(np.fliplr(image))
#     steering_angles.append(-steering_angle)
    
# for img_path_orig, steering_angle in zip(driving_log['right'][high_steering], driving_log['steering'][high_steering]):
#     img_path = update_path(img_path_orig, 'data/IMG/')
#     image = plt.imread(img_path)
#     steering_angle -= 0.25
#     images.append(image)
#     steering_angles.append(steering_angle)
#     images.append(np.fliplr(image))
#     steering_angles.append(-steering_angle)
#     images.append(image)
#     steering_angles.append(steering_angle)
#     images.append(np.fliplr(image))
#     steering_angles.append(-steering_angle)
    
X_train = np.array(images)
y_train = np.array(steering_angles)

In [3]:
from keras.applications import VGG19, VGG16, InceptionV3

#pretrained = VGG19(weights='imagenet', include_top=False, input_shape=(160,320,3))
pretrained = VGG16(weights='imagenet', include_top=False, input_shape=(160,320,3))
pretrained = VGG16(weights='imagenet', include_top=False, input_shape=(80,320,3))


#pretrained = InceptionV3(weights='imagenet', include_top=False, input_shape=(160,320,3))

for layer in pretrained.layers:
    layer.trainable = False

Using TensorFlow backend.
/opt/miniconda3/envs/car2/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/miniconda3/envs/car2/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/miniconda3/envs/car2/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/miniconda3/envs/car2/lib/python3.5/site-pa

/opt/miniconda3/envs/car2/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/miniconda3/envs/car2/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/miniconda3/envs/car2/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/miniconda3/envs/car2/lib/python3.5/site-package

In [7]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Input, Cropping2D, Dropout, Activation, Concatenate
from keras.callbacks import EarlyStopping

stopper = EarlyStopping(monitor='val_loss', min_delta=0.0003, patience=3)

def my_loss_fn(y_true, y_pred):
    weighted_squared_difference = (y_true - y_pred)**2 * np.abs(y_true) # what if true = 0?
    return weighted_squared_difference 

common_dense = Dense(128)

# inp = Input(shape=(160,320,3))
# left = Cropping2D(cropping=((60,20), (0,140)))(inp)
# right = Cropping2D(cropping=((60,20), (140,0)))(inp)
# left = Lambda(lambda x: (x / 255.0) - 0.5)(left)
# right = Lambda(lambda x: (x / 255.0) - 0.5)(right)
# left = pretrained(left)
# right = pretrained(right)
# left = Flatten()(left)
# right = Flatten()(right)
# left = Dropout(rate=0.3)(left)
# right = Dropout(rate=0.3)(right)
# left = common_dense(left)
# right = common_dense(right)
# x = Concatenate(axis=1)([left, right])
# x = Activation('relu')(x)
# x = Dropout(rate=0.3)(x)
# x = Dense(128)(x)
# x = Activation('relu')(x)
# x = Dropout(rate=0.3)(x)
# x = Dense(1)(x)
# model = Model(inputs = inp, outputs=x)

inp = Input(shape=(160,320,3))
x = Cropping2D(cropping=((60,20), (0,0)))(inp)
x = Lambda(lambda x: (x / 255.0) - 0.5)(x)
x = pretrained(x)
x = Flatten()(x)
x = Dropout(rate=0.3)(x)
x = Dense(400)(x)
x = Activation('relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(1)(x)
model = Model(inputs = inp, outputs=x)

model.compile(loss=my_loss_fn, optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=30, batch_size=128, callbacks=[stopper])

model.save('model_30_custom_loss.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 38572 samples, validate on 9644 samples
Epoch 1/30
38572/38572 [==============================] - 79s 2ms/step - loss: 0.2358 - val_loss: 0.0144
Epoch 2/30
38572/38572 [==============================] - 77s 2ms/step - loss: 0.0090 - val_loss: 0.0105
Epoch 3/30
38572/38572 [==============================] - 77s 2ms/step - loss: 0.0074 - val_loss: 0.0106
Epoch 4/30
38572/38572 [==============================] - 77s 2ms/step - loss: 0.0068 - val_loss: 0.0095
Epoch 5/30
38572/38572 [==============================] - 76s 2ms/step - loss: 0.0062 - val_loss: 0.0099
Epoch 6/30
38572/38572 [==============================] - 76s 2ms/step - loss: 0.0057 - val_loss: 0.0095
Epoch 7/30
38572/38572 [==============================] - 76s 2ms/step - loss: 0.0054 - val_loss: 0.0089
Epoch 8/30
38572/38572 [==============================] - 76s 2ms/step - loss: 0.0052 - val_loss: 0.0088
Epoch 9/30
38572/

In [9]:
# model.compile(loss='mse', optimizer='adam')
model.save('model_30_custom_loss.h5')

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 160, 320, 3)  0                                            
__________________________________________________________________________________________________
cropping2d_20 (Cropping2D)      (None, 80, 180, 3)   0           input_19[0][0]                   
__________________________________________________________________________________________________
cropping2d_21 (Cropping2D)      (None, 80, 180, 3)   0           input_19[0][0]                   
__________________________________________________________________________________________________
lambda_20 (Lambda)              (None, 80, 180, 3)   0           cropping2d_20[0][0]              
__________________________________________________________________________________________________
lambda_21 